In [ ]:
from pathlib import Path
from PIL import Image, ImageShow
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.listdir('/content/drive/My Drive/Capstone/irrigation-capstone/data/BigEarthNet/S2A_MSIL2A_20170613T101031_0_52/')

['S2A_MSIL2A_20170613T101031_0_52_B12.tif',
 'S2A_MSIL2A_20170613T101031_0_52_B04.tif',
 'S2A_MSIL2A_20170613T101031_0_52_B07.tif',
 'S2A_MSIL2A_20170613T101031_0_52_B05.tif',
 'S2A_MSIL2A_20170613T101031_0_52_B06.tif',
 'S2A_MSIL2A_20170613T101031_0_52_B11.tif',
 'S2A_MSIL2A_20170613T101031_0_52_B8A.tif',
 'S2A_MSIL2A_20170613T101031_0_52_B02.tif',
 'S2A_MSIL2A_20170613T101031_0_52_labels_metadata.json',
 'S2A_MSIL2A_20170613T101031_0_52_B03.tif',
 'S2A_MSIL2A_20170613T101031_0_52_B09.tif',
 'S2A_MSIL2A_20170613T101031_0_52_B08.tif',
 'S2A_MSIL2A_20170613T101031_0_52_B01.tif']

In [ ]:
# BigEarthNet data definitions 
# https://www.tensorflow.org/datasets/catalog/bigearthnet

# B01: Coastal aerosol; 60m
# B02: Blue; 10m
# B03: Green; 10m
# B04: Red; 10m
# B05: Vegetation red edge; 20m
# B06: Vegetation red edge; 20m
# B07: Vegetation red edge; 20m
# B08: NIR; 10m
# B09: Water vapor; 60m
# B11: SWIR; 20m
# B12: SWIR; 20m
# B8A: Narrow NIR; 20m

In [ ]:
import json
import pandas as pd
data = json.load(open('/content/drive/My Drive/Capstone/irrigation-capstone/data/BigEarthNet/S2A_MSIL2A_20170613T101031_0_52/S2A_MSIL2A_20170613T101031_0_52_labels_metadata.json'))
data

df = pd.DataFrame(data["labels"])
df


,0
0,Sea and ocean


In [ ]:
data['labels']

['Sea and ocean']

In [ ]:
# _LABELS = [
#     'Agro-forestry areas', 'Airports',
#     'Annual crops associated with permanent crops', 'Bare rock',
#     'Beaches, dunes, sands', 'Broad-leaved forest', 'Burnt areas',
#     'Coastal lagoons', 'Complex cultivation patterns', 'Coniferous forest',
#     'Construction sites', 'Continuous urban fabric',
#     'Discontinuous urban fabric', 'Dump sites', 'Estuaries',
#     'Fruit trees and berry plantations', 'Green urban areas',
#     'Industrial or commercial units', 'Inland marshes', 'Intertidal flats',
#     'Land principally occupied by agriculture, with significant areas of '
#     'natural vegetation', 'Mineral extraction sites', 'Mixed forest',
#     'Moors and heathland', 'Natural grassland', 'Non-irrigated arable land',
#     'Olive groves', 'Pastures', 'Peatbogs', 'Permanently irrigated land',
#     'Port areas', 'Rice fields', 'Road and rail networks and associated land',
#     'Salines', 'Salt marshes', 'Sclerophyllous vegetation', 'Sea and ocean',
#     'Sparsely vegetated areas', 'Sport and leisure facilities',
#     'Transitional woodland/shrub', 'Vineyards', 'Water bodies', 'Water courses'
# ]

# _DATA_OPTIONS = ['rgb', 'all']

# # Cr: https://github.com/tensorflow/datasets/blob/master/tensorflow_datasets/image_classification/bigearthnet.py

In [ ]:
data_dir = "/content/drive/My Drive/Capstone/irrigation-capstone/data/BigEarthNet"

In [ ]:
img_list = os.listdir(data_dir)

In [ ]:
label_map = {'Agro-forestry areas':1,
           'Airports':0,
           'Annual crops associated with permanent crops':1,
           'Bare rock':0,
           'Beaches, dunes, sands':0,
           'Broad-leaved forest':0,
           'Burnt areas':0,
           'Coastal lagoons':0,
           'Complex cultivation patterns':1,
           'Coniferous forest':0,
           'Construction sites':0,
           'Continuous urban fabric':0,
           'Discontinuous urban fabric':0,
           'Dump sites':0,
           'Estuaries':0,
           'Fruit trees and berry plantations':1,
           'Green urban areas':0,
           'Industrial or commercial units':0,
           'Inland marshes':0,
           'Intertidal flats':0,
           'Land principally occupied by agriculture,with significant areas of natural vegetation':1,
           'Mineral extraction sites':0,
           'Mixed forest':0,
           'Moors and heathland':0,
           'Natural grassland':0,
           'Non-irrigated arable land':0,
           'Olive groves':1,
           'Pastures':1,
           'Peatbogs':0,
           'Permanently irrigated land':1,
           'Port areas':0,
           'Rice fields':1,
           'Road and rail networks and associated land':0,
           'Salines':0,
           'Salt marshes':0,
           'Sclerophyllous vegetation':0,
           'Sea and ocean':0,
           'Sparsely vegetated areas':0,
           'Sport and leisure facilities':0,
           'Transitional woodland/shrub':0,
           'Vineyards':1,
           'Water bodies':0,
           'Water courses':0
           }

In [ ]:
label_map['Agro-forestry areas']

1

In [ ]:
class Dataset(tf.keras.utils.Sequence):
  def __init__(self, img_list, label_map, shuffle, batch_size):
    self.img_list = img_list
    self.shuffle = shuffle
    self.batch_size = batch_size
    self.label_map = label_map

    self.__on_epoch_end()

  def __len__(self):
    # denote the number of batches in an epoch
    return int(np.floor(len(self.img_list))/self.batch_size)

  def __on_epoch_end(self):
    'create and update index of the images in each epoch'
    self.indexes = np.arange(len(self.img_list))
    if self.shuffle == True:
      np.random.shuffle(self.indexes)

  def __getitem__(self, index):
    indexes = self.indexes[index * self.batch_size: (index + 1) * self.batch_size]
    
    img_list = [self.img_list[i] for i in indexes]
    imgs = [self.__load_img(img_path) for img_path in img_list]
    label = [1 for img_path in img_list]

    #####
    # to add logic to generate label from json and label_map #
    #####

    imgs = np.array(imgs)
    label = np.array(label)
    return imgs, label

  # def __generate_label(img_path, d):
  #   json_path = os.path.join(data_dir, img_path, img_path + '_B02.tif')
  #   for a in B:
  #     if :
  #       return 1
  #   return 0
    
  #   return 1|0

  def __load_img(self, img_path):
    b_path = os.path.join(data_dir, img_path, img_path + '_B02.tif')
    g_path = os.path.join(data_dir, img_path, img_path + '_B03.tif')
    r_path = os.path.join(data_dir, img_path, img_path + '_B04.tif')

    b = np.array(Image.open(b_path))
    g = np.array(Image.open(g_path))
    r = np.array(Image.open(r_path))

    rgb = np.dstack((r,g,b))

    return rgb

  def __preprocess(self, img):

    ######
    ### To add preprocessing logic that is reflective of our EDA / visualization ###
    ### and to allow more remote sensing channels in, other than RGB
    ######
  
    pass

In [ ]:
data_gen = Dataset(img_list, label_map, True, 5)

In [ ]:
data_gen.__getitem__(0)

(array([[[[220, 243, 194],
          [205, 241, 188],
          [212, 243, 198],
          ...,
          [231, 259, 232],
          [244, 259, 233],
          [246, 274, 241]],
 
         [[220, 236, 198],
          [200, 227, 188],
          [204, 245, 201],
          ...,
          [249, 273, 254],
          [251, 281, 241],
          [246, 269, 236]],
 
         [[218, 247, 173],
          [198, 224, 192],
          [206, 220, 204],
          ...,
          [234, 271, 243],
          [260, 266, 246],
          [250, 273, 242]],
 
         ...,
 
         [[211, 247, 193],
          [238, 254, 206],
          [250, 269, 226],
          ...,
          [641, 604, 377],
          [392, 438, 245],
          [227, 328, 128]],
 
         [[212, 248, 188],
          [248, 265, 222],
          [257, 284, 245],
          ...,
          [328, 444, 225],
          [232, 307, 152],
          [153, 278, 103]],
 
         [[224, 265, 212],
          [249, 275, 215],
          [238, 252, 227],
   

In [ ]:
img, label = data_gen.__getitem__(0)

In [ ]:
label.shape

(5,)

In [ ]:
img.shape

(5, 120, 120, 3)

In [ ]:
train_list = img_list[:int(len(img_list)*0.8)]
test_list = img_list[int(len(img_list)*0.8):]

In [ ]:
base_model = tf.keras.applications.resnet_v2.ResNet50V2(include_top=False, weights=None, pooling='avg')
x = base_model.output
x = tf.keras.layers.Dropout(0.5)(x)
out = tf.keras.layers.Dense(1, activation='sigmoid')(x)

In [ ]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=out)

In [ ]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, None, None, 6 0           conv1_conv[0][0]                 
_______________________________________________________________________________________

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
data_train = Dataset(train_list, label_map, True, 5)
data_test = Dataset(test_list, label_map, False, 5)

In [ ]:
model.fit(data_train, validation_data=data_test, epochs=1)

29/29 [==============================] - 35s 1s/step - loss: 1.5047e-04 - accuracy: 1.0000 - val_loss: 0.0054 - val_accuracy: 1.0000


In [ ]:
model.evaluate(data_test, batch_size=64)

7/7 [==============================] - 1s 213ms/step - loss: 0.0054 - accuracy: 1.0000


[0.005369278136640787, 1.0]